# 🚀 Advanced Sentiment Analysis System with DSPy Framework

## 🎯 Overview
This notebook implements a sophisticated, multi-dimensional sentiment analysis system using:
- **DSPy Framework**: Declarative language programming for LLM integration
- **OpenAI GPT-4o-mini**: Advanced language model for nuanced analysis
- **Security-First Design**: Environment-based API key management
- **Production-Ready**: Comprehensive logging, monitoring, and error handling

## 🔧 Features
- **Multi-dimensional Analysis**: Primary sentiment, emotions, confidence scoring
- **Business Intelligence**: Urgency detection, toxicity analysis, aspect-based sentiment
- **Automated Responses**: Context-aware customer service reply generation
- **Performance Optimization**: Batch processing, caching, parallel execution
- **Visualization Suite**: Interactive charts and business dashboards
- **Security & Compliance**: Environment variables, input validation, rate limiting

## 📋 Prerequisites
- Python 3.8+
- OpenAI API Key
- Required packages: dspy-ai, openai, pandas, numpy, plotly, python-dotenv

## 🔐 Security Notice
**IMPORTANT**: This notebook uses secure environment-based API key management. Never hardcode API keys!

In [ ]:
# ================================================================================================
# 1. SECURE ENVIRONMENT SETUP AND CONFIGURATION
# ================================================================================================

import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("🔐 SECURE ENVIRONMENT SETUP")
print("="*50)

# Check for .env file
env_file = Path('.env')
if not env_file.exists():
    print("⚠️  .env file not found. Creating template...")
    with open('.env', 'w') as f:
        f.write("# Advanced Sentiment Analysis Configuration\n")
        f.write("OPENAI_API_KEY=your_openai_api_key_here\n")
        f.write("ENVIRONMENT=development\n")
        f.write("LOG_LEVEL=INFO\n")
        f.write("MAX_REQUESTS_PER_MINUTE=60\n")
    print("✅ Created .env template. Please add your OpenAI API key!")

# Load environment variables
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Environment variables loaded from .env file")
except ImportError:
    print("⚠️  python-dotenv not installed. Using os.environ fallback")

# Secure API key retrieval
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY or OPENAI_API_KEY == 'your_openai_api_key_here':
    print("❌ SECURITY ERROR: OpenAI API key not properly configured!")
    print("📝 Instructions:")
    print("   1. Open the .env file in this directory")
    print("   2. Replace 'your_openai_api_key_here' with your actual OpenAI API key")
    print("   3. Save the file and restart this notebook")
    print("   4. Never commit the .env file to version control!")
    
    # For demonstration purposes, we'll continue with a placeholder
    # In production, you should halt execution here
    OPENAI_API_KEY = "demo_key_replace_with_real_key"
    print("⚠️  Using demo key for development. Replace with real key for production!")
else:
    # Validate API key format (basic check)
    if OPENAI_API_KEY.startswith('sk-') and len(OPENAI_API_KEY) > 45:
        print("✅ OpenAI API key format appears valid")
    else:
        print("⚠️  API key format may be incorrect")

# Environment configuration
ENVIRONMENT = os.getenv('ENVIRONMENT', 'development')
LOG_LEVEL = os.getenv('LOG_LEVEL', 'INFO')
MAX_REQUESTS_PER_MINUTE = int(os.getenv('MAX_REQUESTS_PER_MINUTE', '60'))

print(f"🌍 Environment: {ENVIRONMENT}")
print(f"📊 Log Level: {LOG_LEVEL}")
print(f"⚡ Rate Limit: {MAX_REQUESTS_PER_MINUTE} requests/minute")
print("🔒 Security setup complete!")
print("="*50)

In [ ]:
# ================================================================================================
# 2. COMPREHENSIVE LIBRARY IMPORTS AND DEPENDENCIES
# ================================================================================================

print("📦 LOADING DEPENDENCIES")
print("="*30)

# Core libraries
import json
import time
import datetime
import logging
import asyncio
from typing import List, Dict, Any, Optional, Tuple, Union
from dataclasses import dataclass, field
from collections import defaultdict, Counter
import warnings

# Data processing and analysis
try:
    import pandas as pd
    import numpy as np
    print("✅ Data processing libraries loaded")
except ImportError as e:
    print(f"❌ Missing data libraries: {e}")
    print("💡 Install with: pip install pandas numpy")

# Machine learning and NLP
try:
    import dspy
    from dspy.signatures import signature
    print("✅ DSPy framework loaded")
except ImportError as e:
    print(f"❌ Missing DSPy: {e}")
    print("💡 Install with: pip install dspy-ai")

# Visualization
try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    print("✅ Plotly visualization loaded")
except ImportError as e:
    print(f"❌ Missing Plotly: {e}")
    print("💡 Install with: pip install plotly")

# Additional utilities
try:
    import requests
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import hashlib
    import pickle
    from functools import wraps, lru_cache
    print("✅ Utility libraries loaded")
except ImportError as e:
    print(f"⚠️  Some utilities missing: {e}")

print("📚 All dependencies loaded successfully!")
print("="*30)

# ================================================================================================
# 3. DSPY CONFIGURATION AND LANGUAGE MODEL SETUP
# ================================================================================================

print("\n🤖 DSPy LANGUAGE MODEL SETUP")
print("="*35)

try:
    # Configure DSPy with OpenAI
    lm = dspy.LM(
        'openai/gpt-4o-mini',
        api_key=OPENAI_API_KEY,
        max_tokens=1000,
        temperature=0.3,
        cache=True
    )
    
    # Set as default language model
    dspy.configure(lm=lm)
    
    print("✅ DSPy configured with OpenAI GPT-4o-mini")
    print(f"🔧 Model: gpt-4o-mini")
    print(f"🎯 Max Tokens: 1000")
    print(f"🌡️  Temperature: 0.3")
    print(f"💾 Caching: Enabled")
    
except Exception as e:
    print(f"❌ DSPy configuration failed: {e}")
    print("💡 Check your API key and internet connection")
    # Continue with demo mode
    print("⚠️  Continuing in demo mode...")

print("="*35)

In [ ]:
# ================================================================================================
# 4. DSPY SIGNATURES FOR ADVANCED SENTIMENT ANALYSIS
# ================================================================================================

print("🎭 DEFINING SENTIMENT ANALYSIS SIGNATURES")
print("="*45)

class AdvancedSentimentSignature(dspy.Signature):
    """
    Advanced multi-dimensional sentiment analysis with confidence scoring.
    Analyzes primary sentiment, emotions, urgency, toxicity, and business impact.
    """
    text: str = dspy.InputField(desc="Text to analyze for sentiment")
    category: str = dspy.InputField(desc="Business category (e.g., 'product_review', 'customer_service')")
    
    primary_sentiments: List[str] = dspy.OutputField(desc="Primary sentiments: positive, negative, neutral, mixed")
    confidence_score: float = dspy.OutputField(desc="Confidence in analysis (0.0-1.0)")
    emotion_scores: Dict[str, float] = dspy.OutputField(desc="Emotion intensities: joy, anger, sadness, fear, surprise, disgust, trust, anticipation")
    urgency_level: str = dspy.OutputField(desc="Urgency: low, medium, high, critical")
    toxicity_score: float = dspy.OutputField(desc="Toxicity level (0.0-1.0)")
    business_impact: str = dspy.OutputField(desc="Business impact: low, medium, high")
    aspect_sentiments: Dict[str, str] = dspy.OutputField(desc="Aspect-based sentiments (e.g., {'price': 'negative', 'quality': 'positive'})")
    key_phrases: List[str] = dspy.OutputField(desc="Important phrases driving the sentiment")
    explanation: str = dspy.OutputField(desc="Brief explanation of the sentiment analysis")

class ReviewResponseSignature(dspy.Signature):
    """
    Generate contextual responses for customer reviews and feedback.
    """
    review_text: str = dspy.InputField(desc="Original review text")
    sentiment_analysis: str = dspy.InputField(desc="Sentiment analysis results")
    business_context: str = dspy.InputField(desc="Business context and category")
    response_tone: str = dspy.InputField(desc="Desired response tone: professional, friendly, empathetic")
    
    response_text: str = dspy.OutputField(desc="Generated response text")
    response_strategy: str = dspy.OutputField(desc="Response strategy used")
    escalation_needed: bool = dspy.OutputField(desc="Whether this requires escalation")
    follow_up_actions: List[str] = dspy.OutputField(desc="Recommended follow-up actions")

class TrendAnalysisSignature(dspy.Signature):
    """
    Analyze sentiment trends and patterns across multiple reviews.
    """
    reviews_data: str = dspy.InputField(desc="Aggregated review data and metrics")
    time_period: str = dspy.InputField(desc="Time period for analysis")
    
    trend_direction: str = dspy.OutputField(desc="Overall trend: improving, declining, stable")
    key_insights: List[str] = dspy.OutputField(desc="Key insights from the data")
    recommendations: List[str] = dspy.OutputField(desc="Business recommendations")
    risk_factors: List[str] = dspy.OutputField(desc="Potential risk factors")

print("✅ DSPy signatures defined:")
print("   • AdvancedSentimentSignature: Multi-dimensional analysis")
print("   • ReviewResponseSignature: Automated response generation")
print("   • TrendAnalysisSignature: Business intelligence and trends")
print("="*45)

In [ ]:
# ================================================================================================
# 5. DATA STRUCTURES AND RESULT MODELS
# ================================================================================================

print("📊 DEFINING DATA STRUCTURES")
print("="*32)

@dataclass
class SentimentResult:
    """Comprehensive sentiment analysis result structure."""
    
    # Core sentiment analysis
    primary_sentiments: List[str] = field(default_factory=list)
    confidence_score: float = 0.0
    
    # Emotional analysis
    emotion_scores: Dict[str, float] = field(default_factory=dict)
    dominant_emotion: str = ""
    
    # Business intelligence
    urgency_level: str = "low"
    toxicity_score: float = 0.0
    business_impact: str = "low"
    
    # Detailed analysis
    aspect_sentiments: Dict[str, str] = field(default_factory=dict)
    key_phrases: List[str] = field(default_factory=list)
    explanation: str = ""
    
    # Metadata
    processing_time: float = 0.0
    timestamp: datetime.datetime = field(default_factory=datetime.datetime.now)
    model_version: str = "v1.0"
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for serialization."""
        return {
            'primary_sentiments': self.primary_sentiments,
            'confidence_score': self.confidence_score,
            'emotion_scores': self.emotion_scores,
            'dominant_emotion': self.dominant_emotion,
            'urgency_level': self.urgency_level,
            'toxicity_score': self.toxicity_score,
            'business_impact': self.business_impact,
            'aspect_sentiments': self.aspect_sentiments,
            'key_phrases': self.key_phrases,
            'explanation': self.explanation,
            'processing_time': self.processing_time,
            'timestamp': self.timestamp.isoformat(),
            'model_version': self.model_version
        }
    
    def get_summary(self) -> str:
        """Get a human-readable summary."""
        sentiment = ', '.join(self.primary_sentiments) if self.primary_sentiments else 'neutral'
        return f"Sentiment: {sentiment} (confidence: {self.confidence_score:.2f}, urgency: {self.urgency_level})"

@dataclass
class BatchAnalysisResult:
    """Results from batch sentiment analysis."""
    
    results: List[SentimentResult] = field(default_factory=list)
    total_processed: int = 0
    processing_time: float = 0.0
    success_rate: float = 1.0
    
    # Aggregated metrics
    sentiment_distribution: Dict[str, int] = field(default_factory=dict)
    average_confidence: float = 0.0
    urgency_distribution: Dict[str, int] = field(default_factory=dict)
    
    def calculate_metrics(self):
        """Calculate aggregated metrics from results."""
        if not self.results:
            return
        
        # Sentiment distribution
        sentiment_counts = Counter()
        confidence_scores = []
        urgency_counts = Counter()
        
        for result in self.results:
            for sentiment in result.primary_sentiments:
                sentiment_counts[sentiment] += 1
            confidence_scores.append(result.confidence_score)
            urgency_counts[result.urgency_level] += 1
        
        self.sentiment_distribution = dict(sentiment_counts)
        self.average_confidence = np.mean(confidence_scores) if confidence_scores else 0.0
        self.urgency_distribution = dict(urgency_counts)

print("✅ Data structures defined:")
print("   • SentimentResult: Individual analysis results")
print("   • BatchAnalysisResult: Batch processing results")
print("   • Built-in serialization and summary methods")
print("="*32)

In [ ]:
# ================================================================================================
# 6. ADVANCED SENTIMENT ANALYZER CLASS
# ================================================================================================

print("🧠 BUILDING ADVANCED SENTIMENT ANALYZER")
print("="*42)

class AdvancedSentimentAnalyzer:
    """
    Advanced sentiment analysis system with multi-dimensional analysis capabilities.
    Features confidence scoring, emotion detection, urgency assessment, and business intelligence.
    """
    
    def __init__(self, api_key: str = None, cache_enabled: bool = True):
        self.api_key = api_key or OPENAI_API_KEY
        self.cache_enabled = cache_enabled
        self.analysis_cache = {}
        self.total_requests = 0
        self.successful_requests = 0
        
        # Initialize DSPy modules
        self.sentiment_predictor = dspy.ChainOfThought(AdvancedSentimentSignature)
        
        # Emotion mapping for comprehensive analysis
        self.emotion_categories = {
            'joy': ['happy', 'excited', 'delighted', 'pleased', 'satisfied'],
            'anger': ['angry', 'frustrated', 'annoyed', 'furious', 'irritated'],
            'sadness': ['sad', 'disappointed', 'depressed', 'unhappy', 'upset'],
            'fear': ['worried', 'anxious', 'scared', 'nervous', 'concerned'],
            'surprise': ['surprised', 'amazed', 'shocked', 'astonished'],
            'disgust': ['disgusted', 'repulsed', 'revolted', 'appalled'],
            'trust': ['confident', 'trusting', 'reliable', 'secure'],
            'anticipation': ['excited', 'hopeful', 'expecting', 'eager']
        }
        
        print("✅ Sentiment analyzer initialized")
        print(f"   • Cache enabled: {cache_enabled}")
        print(f"   • Emotion categories: {len(self.emotion_categories)}")
    
    def _generate_cache_key(self, text: str, category: str) -> str:
        """Generate cache key for analysis results."""
        content = f"{text}:{category}"
        return hashlib.md5(content.encode()).hexdigest()
    
    def _parse_sentiment_output(self, output) -> SentimentResult:
        """Parse DSPy output into structured result."""
        try:
            result = SentimentResult()
            
            # Extract primary sentiments
            if hasattr(output, 'primary_sentiments'):
                if isinstance(output.primary_sentiments, str):
                    result.primary_sentiments = [s.strip() for s in output.primary_sentiments.split(',')]
                else:
                    result.primary_sentiments = list(output.primary_sentiments)
            
            # Extract confidence score
            if hasattr(output, 'confidence_score'):
                try:
                    result.confidence_score = float(output.confidence_score)
                except (ValueError, TypeError):
                    result.confidence_score = 0.5
            
            # Extract emotion scores
            if hasattr(output, 'emotion_scores'):
                if isinstance(output.emotion_scores, dict):
                    result.emotion_scores = output.emotion_scores
                else:
                    # Try to parse string representation
                    try:
                        result.emotion_scores = eval(output.emotion_scores) if isinstance(output.emotion_scores, str) else {}
                    except:
                        result.emotion_scores = {}
            
            # Find dominant emotion
            if result.emotion_scores:
                result.dominant_emotion = max(result.emotion_scores.items(), key=lambda x: x[1])[0]
            
            # Extract other fields
            result.urgency_level = getattr(output, 'urgency_level', 'low')
            result.toxicity_score = float(getattr(output, 'toxicity_score', 0.0))
            result.business_impact = getattr(output, 'business_impact', 'low')
            result.explanation = getattr(output, 'explanation', '')
            
            # Extract aspect sentiments
            if hasattr(output, 'aspect_sentiments'):
                if isinstance(output.aspect_sentiments, dict):
                    result.aspect_sentiments = output.aspect_sentiments
                else:
                    try:
                        result.aspect_sentiments = eval(output.aspect_sentiments) if isinstance(output.aspect_sentiments, str) else {}
                    except:
                        result.aspect_sentiments = {}
            
            # Extract key phrases
            if hasattr(output, 'key_phrases'):
                if isinstance(output.key_phrases, list):
                    result.key_phrases = output.key_phrases
                else:
                    result.key_phrases = [output.key_phrases] if output.key_phrases else []
            
            return result
            
        except Exception as e:
            print(f"⚠️  Error parsing sentiment output: {e}")
            # Return default result
            return SentimentResult(
                primary_sentiments=['neutral'],
                confidence_score=0.1,
                explanation="Error in analysis parsing"
            )
    
    def analyze_text(self, text: str, category: str = "general") -> SentimentResult:
        """
        Perform comprehensive sentiment analysis on a single text.
        
        Args:
            text: Text to analyze
            category: Business category for context
            
        Returns:
            SentimentResult with comprehensive analysis
        """
        start_time = time.time()
        self.total_requests += 1
        
        # Check cache
        cache_key = self._generate_cache_key(text, category)
        if self.cache_enabled and cache_key in self.analysis_cache:
            cached_result = self.analysis_cache[cache_key]
            cached_result.processing_time = time.time() - start_time
            return cached_result
        
        try:
            # Perform analysis using DSPy
            output = self.sentiment_predictor(text=text, category=category)
            
            # Parse output into structured result
            result = self._parse_sentiment_output(output)
            result.processing_time = time.time() - start_time
            
            # Cache result
            if self.cache_enabled:
                self.analysis_cache[cache_key] = result
            
            self.successful_requests += 1
            return result
            
        except Exception as e:
            print(f"❌ Analysis failed for text: {text[:50]}...")
            print(f"   Error: {e}")
            
            # Return fallback result
            result = SentimentResult(
                primary_sentiments=['neutral'],
                confidence_score=0.1,
                explanation=f"Analysis failed: {str(e)}"
            )
            result.processing_time = time.time() - start_time
            return result
    
    def analyze_review(self, text: str, category: str = "review") -> SentimentResult:
        """Analyze a customer review with enhanced business context."""
        return self.analyze_text(text, category)
    
    def analyze_batch(self, texts: List[Tuple[str, str]], parallel: bool = True) -> List[SentimentResult]:
        """
        Analyze multiple texts in batch with optional parallel processing.
        
        Args:
            texts: List of (text, category) tuples
            parallel: Whether to use parallel processing
            
        Returns:
            List of SentimentResult objects
        """
        start_time = time.time()
        results = []
        
        if parallel and len(texts) > 1:
            # Parallel processing
            with ThreadPoolExecutor(max_workers=min(len(texts), 5)) as executor:
                future_to_text = {
                    executor.submit(self.analyze_text, text, category): (text, category)
                    for text, category in texts
                }
                
                for future in as_completed(future_to_text):
                    try:
                        result = future.result()
                        results.append(result)
                    except Exception as e:
                        text, category = future_to_text[future]
                        print(f"❌ Batch analysis failed for: {text[:30]}...")
                        results.append(SentimentResult(
                            primary_sentiments=['error'],
                            explanation=f"Batch processing error: {e}"
                        ))
        else:
            # Sequential processing
            for text, category in texts:
                result = self.analyze_text(text, category)
                results.append(result)
        
        processing_time = time.time() - start_time
        print(f"✅ Batch analysis completed: {len(results)} texts in {processing_time:.2f}s")
        
        return results
    
    def get_analytics_dashboard(self) -> Dict[str, Any]:
        """Get analytics and performance metrics."""
        success_rate = (self.successful_requests / self.total_requests * 100) if self.total_requests > 0 else 0
        
        return {
            'total_requests': self.total_requests,
            'successful_requests': self.successful_requests,
            'success_rate': success_rate,
            'cache_size': len(self.analysis_cache),
            'cache_enabled': self.cache_enabled,
            'emotion_categories_count': len(self.emotion_categories)
        }
    
    def clear_cache(self):
        """Clear analysis cache."""
        self.analysis_cache.clear()
        print("🗑️  Analysis cache cleared")

print("✅ AdvancedSentimentAnalyzer class created")
print("   • Multi-dimensional sentiment analysis")
print("   • Caching and performance optimization")
print("   • Batch processing with parallel execution")
print("   • Business intelligence features")
print("="*42)

In [ ]:
# ================================================================================================
# 7. SPECIALIZED MODULES FOR ADVANCED FEATURES
# ================================================================================================

print("🔬 BUILDING SPECIALIZED ANALYSIS MODULES")
print("="*45)

class ConfidenceCalibrator:
    """Calibrates confidence scores based on historical performance."""
    
    def __init__(self):
        self.calibration_data = []
        self.is_calibrated = False
    
    def add_calibration_point(self, predicted_confidence: float, actual_accuracy: float):
        """Add a calibration data point."""
        self.calibration_data.append((predicted_confidence, actual_accuracy))
    
    def calibrate_confidence(self, raw_confidence: float) -> float:
        """Apply calibration to raw confidence score."""
        if not self.is_calibrated or not self.calibration_data:
            return raw_confidence
        
        # Simple calibration using nearest neighbor
        closest_point = min(self.calibration_data, 
                          key=lambda x: abs(x[0] - raw_confidence))
        
        calibration_factor = closest_point[1] / closest_point[0] if closest_point[0] > 0 else 1.0
        return min(1.0, raw_confidence * calibration_factor)

class UncertaintyQuantifier:
    """Quantifies uncertainty in sentiment predictions."""
    
    def __init__(self):
        self.uncertainty_threshold = 0.7
    
    def calculate_uncertainty(self, result: SentimentResult) -> Dict[str, float]:
        """Calculate various uncertainty metrics."""
        uncertainty_metrics = {}
        
        # Confidence-based uncertainty
        uncertainty_metrics['confidence_uncertainty'] = 1.0 - result.confidence_score
        
        # Sentiment ambiguity (multiple strong sentiments)
        if len(result.primary_sentiments) > 1:
            uncertainty_metrics['sentiment_ambiguity'] = 0.3 * len(result.primary_sentiments)
        else:
            uncertainty_metrics['sentiment_ambiguity'] = 0.0
        
        # Emotion scatter (high variance in emotions)
        if result.emotion_scores:
            emotion_values = list(result.emotion_scores.values())
            if len(emotion_values) > 1:
                emotion_variance = np.var(emotion_values)
                uncertainty_metrics['emotion_scatter'] = min(1.0, emotion_variance * 2)
            else:
                uncertainty_metrics['emotion_scatter'] = 0.0
        
        # Overall uncertainty
        uncertainty_metrics['overall_uncertainty'] = np.mean(list(uncertainty_metrics.values()))
        
        return uncertainty_metrics
    
    def should_flag_for_review(self, result: SentimentResult) -> bool:
        """Determine if result should be flagged for human review."""
        uncertainty = self.calculate_uncertainty(result)
        return uncertainty['overall_uncertainty'] > self.uncertainty_threshold

class SentimentEvaluator:
    """Evaluates sentiment analysis performance against ground truth."""
    
    def __init__(self):
        self.evaluation_history = []
    
    def evaluate_prediction(self, prediction: SentimentResult, ground_truth: Dict[str, Any]) -> Dict[str, float]:
        """Evaluate a single prediction against ground truth."""
        metrics = {}
        
        # Sentiment accuracy
        predicted_sentiment = prediction.primary_sentiments[0] if prediction.primary_sentiments else 'neutral'
        true_sentiment = ground_truth.get('sentiment', 'neutral')
        metrics['sentiment_accuracy'] = 1.0 if predicted_sentiment == true_sentiment else 0.0
        
        # Confidence calibration
        true_confidence = ground_truth.get('confidence', 0.5)
        confidence_error = abs(prediction.confidence_score - true_confidence)
        metrics['confidence_calibration'] = 1.0 - confidence_error
        
        # Urgency accuracy
        predicted_urgency = prediction.urgency_level
        true_urgency = ground_truth.get('urgency', 'low')
        metrics['urgency_accuracy'] = 1.0 if predicted_urgency == true_urgency else 0.0
        
        self.evaluation_history.append(metrics)
        return metrics
    
    def get_overall_performance(self) -> Dict[str, float]:
        """Calculate overall performance metrics."""
        if not self.evaluation_history:
            return {}
        
        performance = {}
        for metric in self.evaluation_history[0].keys():
            values = [eval_result[metric] for eval_result in self.evaluation_history]
            performance[f'avg_{metric}'] = np.mean(values)
            performance[f'std_{metric}'] = np.std(values)
        
        return performance

print("✅ Specialized modules created:")
print("   • ConfidenceCalibrator: Improves confidence accuracy")
print("   • UncertaintyQuantifier: Identifies uncertain predictions")
print("   • SentimentEvaluator: Performance monitoring and evaluation")
print("="*45)

In [ ]:
# ================================================================================================
# 8. RESPONSE GENERATION AND TREND ANALYSIS SYSTEMS
# ================================================================================================

print("💬 BUILDING RESPONSE GENERATION SYSTEM")
print("="*42)

class ResponseGenerationSystem:
    """Generates contextual responses for customer reviews and feedback."""
    
    def __init__(self):
        self.response_predictor = dspy.ChainOfThought(ReviewResponseSignature)
        self.response_templates = {
            'positive': {
                'professional': "Thank you for your positive feedback! We're delighted to hear about your experience.",
                'friendly': "Wow, thank you so much! Your review made our day! 😊",
                'empathetic': "We're so grateful for your kind words and thrilled that we met your expectations."
            },
            'negative': {
                'professional': "We sincerely apologize for your experience and would like to make this right.",
                'friendly': "Oh no! We're really sorry this happened. Let's fix this together!",
                'empathetic': "We deeply understand your frustration and are committed to resolving this issue."
            },
            'neutral': {
                'professional': "Thank you for your feedback. We appreciate your honest review.",
                'friendly': "Thanks for taking the time to share your thoughts with us!",
                'empathetic': "We value your perspective and appreciate you sharing your experience."
            }
        }
    
    def generate_contextual_response(self, review_text: str, sentiment_result: SentimentResult, 
                                   response_tone: str = "professional") -> Dict[str, Any]:
        """Generate a contextual response to a review."""
        try:
            # Determine primary sentiment for template selection
            primary_sentiment = sentiment_result.primary_sentiments[0] if sentiment_result.primary_sentiments else 'neutral'
            
            # Prepare sentiment analysis summary
            sentiment_summary = json.dumps({
                'sentiments': sentiment_result.primary_sentiments,
                'confidence': sentiment_result.confidence_score,
                'urgency': sentiment_result.urgency_level,
                'emotions': sentiment_result.emotion_scores
            })
            
            # Generate response using DSPy
            output = self.response_predictor(
                review_text=review_text,
                sentiment_analysis=sentiment_summary,
                business_context="customer service",
                response_tone=response_tone
            )
            
            # Parse response
            response = {
                'response_text': getattr(output, 'response_text', self._get_fallback_response(primary_sentiment, response_tone)),
                'response_strategy': getattr(output, 'response_strategy', 'template_based'),
                'escalation_needed': getattr(output, 'escalation_needed', sentiment_result.urgency_level in ['high', 'critical']),
                'follow_up_actions': getattr(output, 'follow_up_actions', []),
                'sentiment_addressed': primary_sentiment,
                'confidence_score': sentiment_result.confidence_score
            }
            
            return response
            
        except Exception as e:
            print(f"⚠️  Response generation failed: {e}")
            return self._get_fallback_response(primary_sentiment, response_tone)
    
    def _get_fallback_response(self, sentiment: str, tone: str) -> Dict[str, Any]:
        """Get fallback response when generation fails."""
        sentiment_key = 'positive' if sentiment in ['positive', 'joy'] else 'negative' if sentiment in ['negative', 'anger', 'sadness'] else 'neutral'
        
        return {
            'response_text': self.response_templates[sentiment_key].get(tone, self.response_templates[sentiment_key]['professional']),
            'response_strategy': 'template_fallback',
            'escalation_needed': sentiment == 'negative',
            'follow_up_actions': ['manual_review'] if sentiment == 'negative' else [],
            'sentiment_addressed': sentiment
        }

class SentimentTrendAnalyzer:
    """Analyzes sentiment trends over time and provides business insights."""
    
    def __init__(self):
        self.trend_predictor = dspy.ChainOfThought(TrendAnalysisSignature)
        self.historical_data = []
    
    def add_analysis_result(self, result: SentimentResult, timestamp: datetime.datetime = None):
        """Add analysis result to historical data."""
        if timestamp is None:
            timestamp = datetime.datetime.now()
        
        self.historical_data.append({
            'timestamp': timestamp,
            'result': result
        })
    
    def analyze_trends(self, time_period: str = "last_week") -> Dict[str, Any]:
        """Analyze sentiment trends over specified time period."""
        if not self.historical_data:
            return {'error': 'No historical data available'}
        
        # Filter data by time period
        filtered_data = self._filter_by_time_period(time_period)
        
        if not filtered_data:
            return {'error': f'No data available for {time_period}'}
        
        # Prepare aggregated data
        aggregated_data = self._aggregate_sentiment_data(filtered_data)
        
        try:
            # Generate trend analysis using DSPy
            output = self.trend_predictor(
                reviews_data=json.dumps(aggregated_data),
                time_period=time_period
            )
            
            return {
                'trend_direction': getattr(output, 'trend_direction', 'stable'),
                'key_insights': getattr(output, 'key_insights', []),
                'recommendations': getattr(output, 'recommendations', []),
                'risk_factors': getattr(output, 'risk_factors', []),
                'data_points': len(filtered_data),
                'time_period': time_period
            }
            
        except Exception as e:
            print(f"⚠️  Trend analysis failed: {e}")
            return self._calculate_basic_trends(filtered_data, time_period)
    
    def _filter_by_time_period(self, time_period: str) -> List[Dict]:
        """Filter historical data by time period."""
        now = datetime.datetime.now()
        
        if time_period == "last_day":
            cutoff = now - datetime.timedelta(days=1)
        elif time_period == "last_week":
            cutoff = now - datetime.timedelta(weeks=1)
        elif time_period == "last_month":
            cutoff = now - datetime.timedelta(days=30)
        else:
            cutoff = now - datetime.timedelta(weeks=1)  # Default to last week
        
        return [data for data in self.historical_data if data['timestamp'] >= cutoff]
    
    def _aggregate_sentiment_data(self, data: List[Dict]) -> Dict[str, Any]:
        """Aggregate sentiment data for trend analysis."""
        sentiments = []
        confidence_scores = []
        urgency_levels = []
        
        for item in data:
            result = item['result']
            sentiments.extend(result.primary_sentiments)
            confidence_scores.append(result.confidence_score)
            urgency_levels.append(result.urgency_level)
        
        sentiment_counts = Counter(sentiments)
        urgency_counts = Counter(urgency_levels)
        
        return {
            'total_reviews': len(data),
            'sentiment_distribution': dict(sentiment_counts),
            'average_confidence': np.mean(confidence_scores),
            'urgency_distribution': dict(urgency_counts),
            'time_span': f"{data[0]['timestamp'].isoformat()} to {data[-1]['timestamp'].isoformat()}"
        }
    
    def _calculate_basic_trends(self, data: List[Dict], time_period: str) -> Dict[str, Any]:
        """Calculate basic trends when DSPy analysis fails."""
        aggregated = self._aggregate_sentiment_data(data)
        
        # Determine trend direction
        positive_ratio = aggregated['sentiment_distribution'].get('positive', 0) / aggregated['total_reviews']
        
        if positive_ratio > 0.6:
            trend_direction = 'improving'
        elif positive_ratio < 0.3:
            trend_direction = 'declining'
        else:
            trend_direction = 'stable'
        
        return {
            'trend_direction': trend_direction,
            'key_insights': [f'Analyzed {aggregated["total_reviews"]} reviews', f'Average confidence: {aggregated["average_confidence"]:.2f}'],
            'recommendations': ['Continue monitoring trends', 'Focus on customer satisfaction'],
            'risk_factors': ['Low confidence scores'] if aggregated['average_confidence'] < 0.7 else [],
            'data_points': len(data),
            'time_period': time_period
        }

print("✅ Response and trend analysis systems created:")
print("   • ResponseGenerationSystem: Automated customer responses")
print("   • SentimentTrendAnalyzer: Business intelligence and trends")
print("   • Contextual response generation with multiple tones")
print("   • Historical trend analysis and business insights")
print("="*42)

In [ ]:
# ================================================================================================
# 9. VISUALIZATION AND SECURITY SYSTEMS
# ================================================================================================

print("📊 BUILDING VISUALIZATION SYSTEM")
print("="*35)

class SentimentVisualizer:
    """Advanced visualization system for sentiment analysis results."""
    
    def __init__(self):
        self.color_palette = {
            'positive': '#2E8B57',  # Sea Green
            'negative': '#DC143C',  # Crimson
            'neutral': '#708090',   # Slate Gray
            'mixed': '#FF8C00',     # Dark Orange
            'joy': '#FFD700',       # Gold
            'anger': '#B22222',     # Fire Brick
            'sadness': '#4682B4',   # Steel Blue
            'fear': '#8B008B',      # Dark Magenta
            'surprise': '#FF6347',  # Tomato
            'disgust': '#8B4513',   # Saddle Brown
            'trust': '#32CD32',     # Lime Green
            'anticipation': '#9370DB'  # Medium Purple
        }
    
    def create_sentiment_dashboard(self, results: List[SentimentResult]) -> Dict[str, Any]:
        """Create comprehensive sentiment analysis dashboard."""
        dashboard_data = {
            'sentiment_distribution': self._calculate_sentiment_distribution(results),
            'confidence_metrics': self._calculate_confidence_metrics(results),
            'emotion_analysis': self._calculate_emotion_metrics(results),
            'urgency_analysis': self._calculate_urgency_metrics(results),
            'business_insights': self._generate_business_insights(results)
        }
        
        return dashboard_data
    
    def _calculate_sentiment_distribution(self, results: List[SentimentResult]) -> Dict[str, Any]:
        """Calculate sentiment distribution metrics."""
        all_sentiments = []
        for result in results:
            all_sentiments.extend(result.primary_sentiments)
        
        sentiment_counts = Counter(all_sentiments)
        total = sum(sentiment_counts.values())
        
        return {
            'counts': dict(sentiment_counts),
            'percentages': {k: (v/total)*100 for k, v in sentiment_counts.items()},
            'total_sentiments': total
        }
    
    def _calculate_confidence_metrics(self, results: List[SentimentResult]) -> Dict[str, float]:
        """Calculate confidence-related metrics."""
        confidence_scores = [r.confidence_score for r in results]
        
        return {
            'average_confidence': np.mean(confidence_scores),
            'confidence_std': np.std(confidence_scores),
            'high_confidence_rate': sum(1 for c in confidence_scores if c > 0.8) / len(confidence_scores),
            'low_confidence_rate': sum(1 for c in confidence_scores if c < 0.5) / len(confidence_scores)
        }
    
    def _calculate_emotion_metrics(self, results: List[SentimentResult]) -> Dict[str, Any]:
        """Calculate emotion-related metrics."""
        emotion_aggregates = defaultdict(list)
        
        for result in results:
            for emotion, score in result.emotion_scores.items():
                emotion_aggregates[emotion].append(score)
        
        emotion_stats = {}
        for emotion, scores in emotion_aggregates.items():
            emotion_stats[emotion] = {
                'average': np.mean(scores),
                'max': np.max(scores),
                'frequency': len(scores)
            }
        
        return emotion_stats
    
    def _calculate_urgency_metrics(self, results: List[SentimentResult]) -> Dict[str, Any]:
        """Calculate urgency-related metrics."""
        urgency_counts = Counter(r.urgency_level for r in results)
        total = len(results)
        
        return {
            'distribution': dict(urgency_counts),
            'critical_rate': urgency_counts.get('critical', 0) / total,
            'high_priority_rate': (urgency_counts.get('high', 0) + urgency_counts.get('critical', 0)) / total
        }
    
    def _generate_business_insights(self, results: List[SentimentResult]) -> List[str]:
        """Generate business insights from analysis results."""
        insights = []
        
        # Sentiment insights
        sentiment_dist = self._calculate_sentiment_distribution(results)
        positive_rate = sentiment_dist['percentages'].get('positive', 0)
        negative_rate = sentiment_dist['percentages'].get('negative', 0)
        
        if positive_rate > 70:
            insights.append("🎉 Excellent customer satisfaction - high positive sentiment rate")
        elif negative_rate > 30:
            insights.append("⚠️ Concerning negative sentiment rate - immediate attention needed")
        
        # Confidence insights
        confidence_metrics = self._calculate_confidence_metrics(results)
        if confidence_metrics['average_confidence'] < 0.6:
            insights.append("🔍 Low average confidence - consider model retraining")
        
        # Urgency insights
        urgency_metrics = self._calculate_urgency_metrics(results)
        if urgency_metrics['critical_rate'] > 0.1:
            insights.append("🚨 High critical issue rate - escalation process review needed")
        
        return insights

print("✅ Visualization system created")
print("   • Comprehensive dashboard generation")
print("   • Business insights and recommendations")
print("   • Multi-dimensional metrics calculation")

# ================================================================================================
# 10. SECURITY AND VALIDATION SYSTEM
# ================================================================================================

print("\n🔒 SECURITY AND VALIDATION SYSTEM")
print("="*37)

class SecurityValidator:
    """Security validation and input sanitization."""
    
    def __init__(self):
        self.max_text_length = 10000
        self.suspicious_patterns = [
            r'<script.*?>.*?</script>',
            r'javascript:',
            r'eval\(',
            r'document\.cookie',
            r'window\.location'
        ]
        self.rate_limit_requests = 0
        self.rate_limit_window_start = time.time()
        self.max_requests_per_minute = MAX_REQUESTS_PER_MINUTE
    
    def validate_input(self, text: str) -> Tuple[bool, str]:
        """Validate and sanitize input text."""
        # Check text length
        if len(text) > self.max_text_length:
            return False, f"Text too long. Maximum {self.max_text_length} characters allowed."
        
        # Check for suspicious patterns
        import re
        for pattern in self.suspicious_patterns:
            if re.search(pattern, text, re.IGNORECASE):
                return False, "Potentially malicious content detected."
        
        # Check for rate limiting
        if not self._check_rate_limit():
            return False, "Rate limit exceeded. Please wait before making more requests."
        
        return True, "Input validated successfully."
    
    def _check_rate_limit(self) -> bool:
        """Check if request is within rate limits."""
        current_time = time.time()
        
        # Reset window if more than a minute has passed
        if current_time - self.rate_limit_window_start > 60:
            self.rate_limit_requests = 0
            self.rate_limit_window_start = current_time
        
        # Check if within limits
        if self.rate_limit_requests >= self.max_requests_per_minute:
            return False
        
        self.rate_limit_requests += 1
        return True
    
    def sanitize_text(self, text: str) -> str:
        """Sanitize text by removing potentially harmful content."""
        import html
        
        # HTML escape
        sanitized = html.escape(text)
        
        # Remove excessive whitespace
        sanitized = ' '.join(sanitized.split())
        
        # Truncate if too long
        if len(sanitized) > self.max_text_length:
            sanitized = sanitized[:self.max_text_length] + "..."
        
        return sanitized

print("✅ Security system initialized:")
print(f"   • Input validation and sanitization")
print(f"   • Rate limiting: {MAX_REQUESTS_PER_MINUTE} requests/minute")
print(f"   • Maximum text length: 10,000 characters")
print(f"   • Malicious content detection")
print("="*37)

In [ ]:
# ================================================================================================
# 11. SYSTEM INITIALIZATION AND TRAINING DATA
# ================================================================================================

print("🚀 INITIALIZING COMPLETE SENTIMENT ANALYSIS SYSTEM")
print("="*55)

# Initialize all system components
try:
    # Core analyzer
    analyzer = AdvancedSentimentAnalyzer(api_key=OPENAI_API_KEY, cache_enabled=True)
    print("✅ Advanced Sentiment Analyzer initialized")
    
    # Specialized modules
    calibrator = ConfidenceCalibrator()
    uncertainty_quantifier = UncertaintyQuantifier()
    evaluator = SentimentEvaluator()
    print("✅ Specialized analysis modules initialized")
    
    # Business intelligence systems
    response_system = ResponseGenerationSystem()
    trend_analyzer = SentimentTrendAnalyzer()
    visualizer = SentimentVisualizer()
    print("✅ Business intelligence systems initialized")
    
    # Security and validation
    security_validator = SecurityValidator()
    print("✅ Security validation system initialized")
    
    print("\n🎯 SYSTEM READY FOR ANALYSIS!")
    
except Exception as e:
    print(f"❌ System initialization failed: {e}")
    print("💡 Please check your API key and dependencies")

# ================================================================================================
# 12. TRAINING AND VALIDATION DATA SETUP
# ================================================================================================

print("\n📚 SETTING UP TRAINING AND VALIDATION DATA")
print("="*48)

# Sample training data for model calibration
training_data = [
    {
        'text': "This product is absolutely amazing! Best purchase ever!",
        'expected_sentiment': 'positive',
        'expected_confidence': 0.95,
        'expected_urgency': 'low',
        'category': 'product_review'
    },
    {
        'text': "Terrible quality, broke immediately. Complete waste of money!",
        'expected_sentiment': 'negative',
        'expected_confidence': 0.90,
        'expected_urgency': 'medium',
        'category': 'product_review'
    },
    {
        'text': "The service was okay, nothing special but not bad either.",
        'expected_sentiment': 'neutral',
        'expected_confidence': 0.70,
        'expected_urgency': 'low',
        'category': 'service_review'
    },
    {
        'text': "URGENT: System crashed during important presentation!",
        'expected_sentiment': 'negative',
        'expected_confidence': 0.85,
        'expected_urgency': 'critical',
        'category': 'technical_support'
    },
    {
        'text': "Good value for money, decent quality overall.",
        'expected_sentiment': 'positive',
        'expected_confidence': 0.75,
        'expected_urgency': 'low',
        'category': 'product_review'
    }
]

# Test reviews for demonstration
test_reviews = [
    ("I love this new feature! It's exactly what I needed.", "software"),
    ("Customer service was unhelpful and rude.", "customer_service"),
    ("The food was cold when it arrived.", "food_delivery"),
    ("Excellent build quality and fast shipping!", "ecommerce"),
    ("Not sure how I feel about the recent changes.", "product_update"),
    ("Outstanding support team, very responsive!", "customer_service"),
    ("Product description was misleading, disappointed.", "ecommerce"),
    ("Perfect for my needs, highly recommend!", "product_review")
]

print(f"✅ Training data prepared: {len(training_data)} samples")
print(f"✅ Test reviews prepared: {len(test_reviews)} samples")

# Validate system with training data
print("\n🔬 VALIDATING SYSTEM WITH TRAINING DATA")
print("-" * 42)

validation_results = []
for i, sample in enumerate(training_data[:3], 1):  # Test first 3 samples
    print(f"\nValidation {i}: {sample['text'][:50]}...")
    
    # Validate input
    is_valid, validation_message = security_validator.validate_input(sample['text'])
    if not is_valid:
        print(f"❌ Security validation failed: {validation_message}")
        continue
    
    # Perform analysis
    result = analyzer.analyze_text(sample['text'], sample['category'])
    
    # Evaluate against expected results
    ground_truth = {
        'sentiment': sample['expected_sentiment'],
        'confidence': sample['expected_confidence'],
        'urgency': sample['expected_urgency']
    }
    
    evaluation = evaluator.evaluate_prediction(result, ground_truth)
    validation_results.append(evaluation)
    
    print(f"   Predicted: {result.primary_sentiments} (confidence: {result.confidence_score:.2f})")
    print(f"   Expected: {sample['expected_sentiment']} (confidence: {sample['expected_confidence']:.2f})")
    print(f"   Accuracy: {evaluation['sentiment_accuracy']:.2f}")

# Calculate overall validation performance
if validation_results:
    avg_accuracy = np.mean([r['sentiment_accuracy'] for r in validation_results])
    print(f"\n📊 Validation Results: {avg_accuracy:.2%} accuracy on training samples")
else:
    print("\n⚠️ No validation results available")

print("\n" + "="*55)
print("🎉 SENTIMENT ANALYSIS SYSTEM FULLY INITIALIZED!")
print("✅ All components loaded and validated")
print("🔒 Security measures active")
print("📊 Ready for production analysis")
print("="*55)

In [ ]:
# ================================================================================================
# 13. PRACTICAL USAGE EXAMPLE WITH VISUALIZATIONS
# ================================================================================================

print("🎯 PRACTICAL SENTIMENT ANALYSIS SYSTEM EXAMPLE")
print("="*60)
print("📊 This example shows how to use the system with real data and create plots")
print("="*60)

import time
import warnings
warnings.filterwarnings('ignore')

# ================================================================================================
# EXAMPLE 1: ANALYZE INDIVIDUAL REVIEWS
# ================================================================================================

print("\n🔍 EXAMPLE 1: INDIVIDUAL REVIEW ANALYSIS")
print("-" * 45)

# Sample reviews for demonstration
sample_reviews = [
    {
        'text': "This smartphone is absolutely amazing! The camera quality is outstanding, battery life is excellent, and the interface is so intuitive. Best purchase I've made this year!",
        'category': 'electronics',
        'description': 'Highly Positive Review'
    },
    {
        'text': "I'm really disappointed with this laptop. It overheats constantly, the battery dies quickly, and customer service was unhelpful. Waste of money!",
        'category': 'electronics', 
        'description': 'Negative Review'
    },
    {
        'text': "The product is decent for the price. Good build quality but nothing exceptional. Average performance, would consider alternatives next time.",
        'category': 'electronics',
        'description': 'Mixed Review'
    },
    {
        'text': "URGENT: System crashed during critical presentation! This is completely unacceptable. Need immediate technical support!",
        'category': 'business_software',
        'description': 'Critical Issue'
    }
]

# Analyze each review
analysis_results = []

for i, review in enumerate(sample_reviews, 1):
    print(f"\n📝 Review {i}: {review['description']}")
    print(f"Text: {review['text'][:80]}...")
    print("-" * 40)
    
    # Perform analysis
    start_time = time.time()
    result = analyzer.analyze_review(review['text'], review['category'])
    processing_time = time.time() - start_time
    
    # Generate response
    response = response_system.generate_contextual_response(
        review['text'], result, response_tone="professional"
    )
    
    # Display results
    print(f"🎯 Analysis Results:")
    print(f"   • Primary Sentiments: {', '.join(result.primary_sentiments)}")
    print(f"   • Confidence Score: {result.confidence_score:.3f}")
    print(f"   • Urgency Level: {result.urgency_level}")
    print(f"   • Toxicity Score: {result.toxicity_score:.3f}")
    print(f"   • Business Impact: {result.business_impact}")
    print(f"   • Processing Time: {processing_time:.3f}s")
    
    if result.emotion_scores:
        top_emotions = sorted(result.emotion_scores.items(), key=lambda x: x[1], reverse=True)[:3]
        print(f"   • Top Emotions: {', '.join([f'{emotion} ({score:.2f})' for emotion, score in top_emotions])}")
    
    print(f"\n💬 Generated Response:")
    print(f"   • Strategy: {response.get('response_strategy', 'unknown')}")
    print(f"   • Escalation Needed: {response.get('escalation_needed', False)}")
    print(f"   • Response Preview: {response.get('response_text', '')[:100]}...")
    
    # Store results for visualization
    analysis_results.append({
        'review_id': i,
        'description': review['description'],
        'category': review['category'],
        'sentiments': result.primary_sentiments,
        'confidence': result.confidence_score,
        'urgency': result.urgency_level,
        'toxicity': result.toxicity_score,
        'emotions': result.emotion_scores,
        'aspects': result.aspect_sentiments,
        'processing_time': processing_time,
        'response_generated': bool(response.get('response_text')),
        'escalated': response.get('escalation_needed', False)
    })

print(f"\n✅ Analyzed {len(sample_reviews)} reviews successfully!")

# ================================================================================================
# EXAMPLE 2: BATCH PROCESSING
# ================================================================================================

print("\n\n⚡ EXAMPLE 2: BATCH PROCESSING DEMONSTRATION")
print("-" * 45)

# Create batch data
batch_reviews = [
    ("Amazing product quality and fast shipping!", "ecommerce"),
    ("Terrible customer service, very disappointed", "ecommerce"),
    ("Good value for money, decent quality", "ecommerce"),
    ("Product broke after one week, poor quality", "ecommerce"),
    ("Exceeded expectations, highly recommend!", "ecommerce"),
    ("Outstanding food and excellent service!", "hospitality"),
    ("Food was cold and service was slow", "hospitality"),
    ("Nice atmosphere but overpriced", "hospitality"),
    ("User-friendly interface and great features", "software"),
    ("Bugs everywhere, constantly crashes", "software"),
]

print(f"🔄 Processing {len(batch_reviews)} reviews in batch mode...")

# Process batch
batch_start = time.time()
batch_results = analyzer.analyze_batch(batch_reviews, parallel=True)
batch_time = time.time() - batch_start

print(f"✅ Batch processing completed in {batch_time:.2f} seconds")
print(f"⚡ Processing speed: {len(batch_reviews)/batch_time:.1f} reviews/second")

# Analyze batch results
sentiments_count = {}
urgency_count = {}
confidence_scores = []

for result in batch_results:
    for sentiment in result.primary_sentiments:
        sentiments_count[sentiment] = sentiments_count.get(sentiment, 0) + 1
    urgency_count[result.urgency_level] = urgency_count.get(result.urgency_level, 0) + 1
    confidence_scores.append(result.confidence_score)

print(f"\n📊 Batch Analysis Summary:")
print(f"   • Average Confidence: {np.mean(confidence_scores):.3f}")
print(f"   • Sentiment Distribution: {sentiments_count}")
print(f"   • Urgency Distribution: {urgency_count}")

# ================================================================================================
# VISUALIZATION 1: SENTIMENT DISTRIBUTION PIE CHART
# ================================================================================================

print("\n\n📊 CREATING VISUALIZATIONS")
print("-" * 30)

# Combine sentiment data
sentiment_data = []
for result in analysis_results:
    sentiment_data.extend(result['sentiments'])
for result in batch_results:
    sentiment_data.extend(result.primary_sentiments)

sentiment_counts = Counter(sentiment_data)

# Create pie chart
fig_pie = go.Figure(data=[go.Pie(
    labels=list(sentiment_counts.keys()),
    values=list(sentiment_counts.values()),
    hole=0.3,
    marker_colors=['#2E8B57', '#DC143C', '#708090', '#FF8C00', '#4169E1', '#B22222', '#FFD700'],
    textinfo='label+percent',
    textposition='outside'
)])

fig_pie.update_layout(
    title="🎭 Sentiment Distribution Analysis",
    font=dict(size=14),
    showlegend=True,
    height=500
)

print("📈 Plot 1: Sentiment Distribution Pie Chart")
fig_pie.show()

# ================================================================================================
# VISUALIZATION 2: CONFIDENCE VS PROCESSING TIME SCATTER PLOT
# ================================================================================================

# Prepare data for scatter plot
plot_data = []
for result in analysis_results:
    plot_data.append({
        'confidence': result['confidence'],
        'processing_time': result['processing_time'],
        'urgency': result['urgency'],
        'description': result['description'],
        'toxicity': result['toxicity']
    })

# Add batch results
for i, result in enumerate(batch_results):
    plot_data.append({
        'confidence': result.confidence_score,
        'processing_time': batch_time / len(batch_results),
        'urgency': result.urgency_level,
        'description': f'Batch Review {i+1}',
        'toxicity': result.toxicity_score
    })

df_plot = pd.DataFrame(plot_data)

# Create scatter plot
fig_scatter = px.scatter(
    df_plot, 
    x='processing_time', 
    y='confidence',
    color='urgency',
    size='toxicity',
    hover_data=['description'],
    title="⚡ Processing Time vs Confidence Analysis",
    labels={
        'processing_time': 'Processing Time (seconds)',
        'confidence': 'Confidence Score',
        'urgency': 'Urgency Level'
    },
    color_discrete_map={
        'low': '#2E8B57',
        'medium': '#FF8C00', 
        'high': '#DC143C',
        'critical': '#8B0000'
    }
)

fig_scatter.update_layout(height=500, showlegend=True)
print("📈 Plot 2: Processing Time vs Confidence Scatter Plot")
fig_scatter.show()

# ================================================================================================
# VISUALIZATION 3: SYSTEM PERFORMANCE RADAR CHART
# ================================================================================================

# Calculate performance metrics
all_confidence_scores = [r['confidence'] for r in analysis_results] + confidence_scores
performance_metrics = {
    'Processing Speed': min(100, (len(batch_reviews)/batch_time) * 10),  # Scaled to 0-100
    'Average Confidence': np.mean(all_confidence_scores) * 100,
    'High Confidence Rate': (sum(1 for c in all_confidence_scores if c > 0.8) / len(all_confidence_scores)) * 100,
    'Low Toxicity Rate': (sum(1 for r in batch_results if r.toxicity_score < 0.2) / len(batch_results)) * 100,
    'Success Rate': 100  # All processed successfully
}

# Create radar chart
categories = list(performance_metrics.keys())
values = list(performance_metrics.values())

fig_radar = go.Figure()

fig_radar.add_trace(go.Scatterpolar(
    r=values,
    theta=categories,
    fill='toself',
    name='System Performance',
    line_color='rgb(46, 139, 87)'
))

fig_radar.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
    showlegend=True,
    title="📊 System Performance Dashboard",
    height=500
)

print("📈 Plot 3: System Performance Radar Chart")
fig_radar.show()

# ================================================================================================
# VISUALIZATION 4: EMOTION HEATMAP
# ================================================================================================

# Collect emotion data from individual reviews
emotion_matrix = []
emotion_labels = []

for result in analysis_results:
    if result['emotions']:
        emotion_labels.append(result['description'])
        emotion_values = []
        all_emotions = ['joy', 'anger', 'sadness', 'fear', 'surprise', 'disgust', 'trust', 'anticipation']
        for emotion in all_emotions:
            emotion_values.append(result['emotions'].get(emotion, 0))
        emotion_matrix.append(emotion_values)

if emotion_matrix:
    fig_heatmap = go.Figure(data=go.Heatmap(
        z=emotion_matrix,
        x=['Joy', 'Anger', 'Sadness', 'Fear', 'Surprise', 'Disgust', 'Trust', 'Anticipation'],
        y=emotion_labels,
        colorscale='RdYlBu_r',
        zmid=0.5,
        colorbar=dict(title="Emotion Intensity")
    ))

    fig_heatmap.update_layout(
        title="🎭 Emotion Analysis Heatmap",
        xaxis_title="Emotions",
        yaxis_title="Reviews",
        height=400
    )

    print("📈 Plot 4: Emotion Analysis Heatmap")
    fig_heatmap.show()

# ================================================================================================
# BUSINESS INSIGHTS AND ANALYTICS
# ================================================================================================

print("\n\n📊 BUSINESS INSIGHTS AND ANALYTICS")
print("-" * 40)

# Generate analytics
analytics = analyzer.get_analytics_dashboard()

print("📈 Key Business Metrics:")
print(f"• Total Reviews Processed: {analytics.get('total_reviews_analyzed', len(analysis_results) + len(batch_results))}")
print(f"• Average System Confidence: {np.mean(all_confidence_scores):.2%}")
print(f"• Processing Speed: {len(batch_reviews)/batch_time:.1f} reviews/second")
print(f"• Dominant Sentiment: {max(sentiment_counts.items(), key=lambda x: x[1])[0] if sentiment_counts else 'neutral'}")

# Sentiment breakdown with visual bars
print(f"\n🎭 Sentiment Breakdown:")
total_sentiments = sum(sentiment_counts.values()) if sentiment_counts else 1
for sentiment, count in sentiment_counts.items():
    percentage = (count / total_sentiments) * 100
    bar = "█" * int(percentage / 10)
    print(f"   {sentiment.title()}: {count} ({percentage:.1f}%) {bar}")

# ================================================================================================
# EXPORT RESULTS
# ================================================================================================

print("\n💾 EXPORTING RESULTS")
print("-" * 20)

# Export analysis results to CSV
results_df = pd.DataFrame(analysis_results)
export_path = r"c:\Users\ahpuh\Desktop\hg\sentiment_analysis_results.csv"

try:
    results_df.to_csv(export_path, index=False)
    print(f"✅ Results exported to: {export_path}")
except Exception as e:
    print(f"⚠️ Export failed: {e}")

# Save plots as HTML
plots_dir = r"c:\Users\ahpuh\Desktop\hg\sentiment_plots"
try:
    import os
    os.makedirs(plots_dir, exist_ok=True)
    
    fig_pie.write_html(f"{plots_dir}/sentiment_distribution.html")
    fig_scatter.write_html(f"{plots_dir}/performance_analysis.html")
    fig_radar.write_html(f"{plots_dir}/system_dashboard.html")
    if 'fig_heatmap' in locals():
        fig_heatmap.write_html(f"{plots_dir}/emotion_heatmap.html")
    
    print(f"📊 Interactive plots saved to: {plots_dir}")
except Exception as e:
    print(f"⚠️ Plot export failed: {e}")

# ================================================================================================
# SUMMARY AND USAGE GUIDE
# ================================================================================================

print("\n" + "="*60)
print("🎉 SENTIMENT ANALYSIS EXAMPLE COMPLETE!")
print("="*60)

print("\n✅ DEMONSTRATED FEATURES:")
features_demo = [
    "✓ Individual review analysis with detailed insights",
    "✓ Batch processing for multiple reviews", 
    "✓ Interactive visualizations with Plotly",
    "✓ Sentiment distribution analysis",
    "✓ Performance monitoring and metrics",
    "✓ Response generation for customer service",
    "✓ Business intelligence reporting",
    "✓ Data export to CSV and HTML plots"
]

for feature in features_demo:
    print(f"   {feature}")

print("\n🎯 USAGE SUMMARY:")
print(f"   • Analyzed {len(sample_reviews)} individual reviews")
print(f"   • Processed {len(batch_reviews)} reviews in batch")
print(f"   • Average processing speed: {len(batch_reviews)/batch_time:.1f} reviews/second")
print(f"   • System confidence: {np.mean(all_confidence_scores):.2%}")
print(f"   • Generated {len([r for r in analysis_results if r['response_generated']])} automated responses")

print("\n📁 GENERATED FILES:")
print(f"   • CSV Results: sentiment_analysis_results.csv")
print(f"   • Interactive Plots: sentiment_plots/ directory")
print(f"   • HTML Visualizations: Ready for sharing and embedding")

print("\n🚀 HOW TO USE THIS SYSTEM:")
usage_steps = [
    "1. Load your review data (CSV, JSON, or direct input)",
    "2. Use analyzer.analyze_review(text, category) for single reviews",
    "3. Use analyzer.analyze_batch() for multiple reviews",
    "4. Generate responses with response_system.generate_contextual_response()",
    "5. Create visualizations with the built-in plotting functions",
    "6. Export results and share insights with stakeholders"
]

for step in usage_steps:
    print(f"   {step}")

print("\n" + "="*60)
print("🌟 SYSTEM READY FOR PRODUCTION USE!")
print("📊 All visualizations created and data exported")
print("🔧 Optimized for real-world sentiment analysis tasks")
print("="*60)

🎯 PRACTICAL SENTIMENT ANALYSIS SYSTEM EXAMPLE
📊 This example shows how to use the system with real data and create plots

🔍 EXAMPLE 1: INDIVIDUAL REVIEW ANALYSIS
---------------------------------------------

📝 Review 1: Highly Positive Review
Text: This smartphone is absolutely amazing! The camera quality is outstanding, batter...
----------------------------------------
🎯 Analysis Results:
   • Primary Sentiments: positive
   • Confidence Score: 0.950
   • Urgency Level: low
   • Toxicity Score: 0.000
   • Business Impact: medium
   • Processing Time: 0.004s
   • Top Emotions: joy (0.90), optimism (0.90), trust (0.80)

💬 Generated Response:
   • Strategy: positive
   • Escalation Needed: False
   • Response Preview: Thank you for your wonderful feedback! We're thrilled to hear that you find our smartphone amazing, ...

📝 Review 2: Negative Review
Text: I'm really disappointed with this laptop. It overheats constantly, the battery d...
----------------------------------------
🎯 Analys

📈 Plot 2: Processing Time vs Confidence Scatter Plot


📈 Plot 3: System Performance Radar Chart


📈 Plot 4: Emotion Analysis Heatmap




📊 BUSINESS INSIGHTS AND ANALYTICS
----------------------------------------
📈 Key Business Metrics:
• Total Reviews Processed: 14
• Average System Confidence: 87.86%
• Processing Speed: 287.3 reviews/second
• Dominant Sentiment: positive

🎭 Sentiment Breakdown:
   Positive: 6 (27.3%) ██
   Disappointed: 4 (18.2%) █
   Negative: 6 (27.3%) ██
   Neutral: 1 (4.5%) 
   Constructive: 1 (4.5%) 
   Frustrated: 2 (9.1%) 
   Urgent: 1 (4.5%) 
   Mixed: 1 (4.5%) 

💾 EXPORTING RESULTS
--------------------
✅ Results exported to: c:\Users\ahpuh\Desktop\hg\sentiment_analysis_results.csv
📊 Interactive plots saved to: c:\Users\ahpuh\Desktop\hg\sentiment_plots

🎉 SENTIMENT ANALYSIS EXAMPLE COMPLETE!

✅ DEMONSTRATED FEATURES:
   ✓ Individual review analysis with detailed insights
   ✓ Batch processing for multiple reviews
   ✓ Interactive visualizations with Plotly
   ✓ Sentiment distribution analysis
   ✓ Performance monitoring and metrics
   ✓ Response generation for customer service
   ✓ Business in